In [38]:
import pandas as pd

df = pd.read_csv('data.csv')  # read df from csv file

df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H;%M:%S')
df = df.drop(columns=['Date', 'Time'])
df.set_index('Datetime', inplace=True)

# using month and hour as features
df['Month'], df['Day'], df['Hour']  = df.index.month, df.index.day, df.index.hour 
    

In [40]:
from System import Plant
from Policy import Policy
import matplotlib.pyplot as plt

plant = Plant(dt=1/60)
plant_comp = Plant(dt=1/60)
plant.reset(init_battery=0, init_month=10, init_day=1, init_hour=0)
plant_comp.reset(init_battery=0, init_month=10, init_day=1, init_hour=0)
policy = Policy()

control = []
battery = []

for data in df[:1].itertuples():
    # Index=Timestamp('2007-10-01 00:00:00'), Load=1.324, Month=10, Day=1, Hour=0
    u, w_hat = policy.select_action(plant)
    control.append(u)
    print(u, w_hat)
    battery.append(plant.battery.c)
    plant.step(u, data[1])
    plant_comp.step(0, data[1])
print("Peak load:", plant.x, "Peak load without control:", plant_comp.x)
plt.plot(control)

TypeError: 'float' object is not iterable